# Example of using OpenAI streaming with Guardrails


In [1]:
# Few imports and global variables
from rich import print
import guardrails as gd
import openai
from IPython.display import clear_output
import time
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

/Users/karanacharya/.pyenv/versions/3.11.6/envs/guard-venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


### 1. For structured JSON output


#### Define the prompt and output schema


In [2]:
from pydantic import BaseModel, Field
from typing import List
from guardrails.validators import (
    ValidRange,
    UpperCase,
    LowerCase,
    OneLine,
)

prompt = """
Given the following doctor's notes about a patient, please extract a dictionary that contains the patient's information.

${doctors_notes}

${gr.complete_json_suffix_v2}
"""

doctors_notes = """152 y/o female with chronic macular rash to face and hair, worse in beard, eyebrows and nares.
The rash is itchy, flaky and slightly scaly. Moderate response to OTC steroid cream. Patient has been using cream for 2 weeks and also suffers from diabetes."""


class Symptom(BaseModel):
    symptom: str = Field(description="Symptom that a patient is experiencing")
    affected_area: str = Field(
        description="What part of the body the symptom is affecting",
        validators=[
            LowerCase(on_fail="fix"),
        ],
    )


class Medication(BaseModel):
    medication: str = Field(
        description="Name of the medication the patient is taking",
        validators=[UpperCase(on_fail="fix")],
    )
    response: str = Field(description="How the patient is responding to the medication")


class PatientInfo(BaseModel):
    gender: str = Field(description="Patient's gender")
    age: int = Field(
        description="Patient's age",
        validators=[ValidRange(min=0, max=100, on_fail="fix")],
    )
    symptoms: List[Symptom] = Field(
        description="Symptoms that the patient is currently experiencing. Each symptom should be classified into  separate item in the list."
    )
    current_meds: List[Medication] = Field(
        description="Medications the patient is currently taking and their response"
    )
    miscellaneous: str = Field(
        description="Any other information that is relevant to the patient's health; something that doesn't fit into the other categories.",
        validators=[LowerCase(on_fail="fix"), OneLine(on_fail="fix")],
    )

#### Create the Guard object


In [3]:
guard = gd.Guard.from_pydantic(output_class=PatientInfo, prompt=prompt)

##### Example 1: No streaming

By default, the `stream` parameter is set to `False`


In [4]:
# Wrap the OpenAI API call with the `guard` object
raw_llm_output, validated_output, *rest = guard(
    openai.completions.create,
    prompt_params={"doctors_notes": doctors_notes},
    engine="text-davinci-003",
    max_tokens=1024,
    temperature=0.3,
)

# Print the validated output from the LLM
print(validated_output)

{
    'gender': 'female',
    'age': 100,
    'symptoms': [
        {'symptom': 'macular rash', 'affected_area': 'face and hair, worse in beard, eyebrows and nares'},
        {
            'symptom': 'itchy, flaky and slightly scaly',
            'affected_area': 'face and hair, worse in beard, eyebrows and nares'
        }
    ],
    'current_meds': [{'medication': 'OTC STEROID CREAM', 'response': 'moderate response'}],
    'miscellaneous': 'suffers from diabetes; using cream for 2 weeks'
}

In [5]:
# Let's see the logs
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Given the following doctor's notes about a patient, please extract a dictionary that contains the       │ │
    │ │ patient's information.                                                                                  │ │
    │ │                                                                                                         │ │
    │ │ 152 y/o female with chronic macular rash to face and hair, worse in beard, eyebrows and nares.          │ │
    │ │ The rash is itchy, flaky and slightly scaly. Moderate response to OTC steroid cream. Patient has been   │ │
    │ │ using cream for 2 weeks and also suffers from diabetes.                                                 │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <string name="gender" description="Patient's gender"/>                                              │ │
    │ │     <integer name="age" description="Patient's age" format="valid-range: min=0 max=100"/>               │ │
    │ │     <list name="symptoms" description="Symptoms that the patient is currently experiencing. Each        │ │
    │ │ symptom should be classified into  separate item in the list.">                                         │ │
    │ │         <object>                                                                                        │ │
    │ │             <string name="symptom" description="Symptom that a patient is experiencing"/>               │ │
    │ │             <string name="affected_area" description="What part of the body the symptom is affecting"   │ │
    │ │ format="lower-case"/>                                                                                   │ │
    │ │         </object>                                                                                       │ │
    │ │     </list>                                                                                             │ │
    │ │     <list name="current_meds" description="Medications the patient is currently taking and their        │ │
    │ │ response">                                                                                              │ │
    │ │         <object>                                                                                        │ │
    │ │             <string name="medication" description="Name of the medication the patient is taking"        │ │
    │ │ format="upper-case"/>                                                                                   │ │
    │ │             <string name="response" description="How the patient is responding to the medication"/>     │ │
    │ │         </object>                                                                                       │ │
    │ │     </list>                                                                                             │ │
    │ │     <string name="miscellaneous" description="Any other information that is relevant to the patient's   │ │
    │ │ health; something that doesn't fit into the

##### Example 2: Streaming

Set the `stream` parameter to `True`


In [6]:
# Wrap the OpenAI API call with the `guard` object
fragment_generator = guard(
    openai.completions.create,
    prompt_params={"doctors_notes": doctors_notes},
    engine="text-davinci-003",
    max_tokens=1024,
    temperature=0,
    stream=True,
)


for op in fragment_generator:
    clear_output(wait=True)
    print(op)
    time.sleep(0.5)

Raw LLM response:
{
    "gender": "female",
    "age": 152,
    "symptoms": [
        {
            "symptom": "macular rash",
            "affected_area": "face and hair, worse in beard, eyebrows and nares"
        },
        {
            "symptom": "itchy, flaky and slightly scaly",
            "affected_area": ""
        }
    ],
    "current_meds": [
        {
            "medication": "OTC steroid cream",
            "response": "moderate response"
        }
    ],
    "miscellaneous": "patient has been using cream for 2 weeks and also suffers from diabetes"
}

Validated response:
{
    "gender": "female",
    "age": 100,
    "symptoms": [
        {
            "symptom": "macular rash",
            "affected_area": "face and hair, worse in beard, eyebrows and nares"
        },
        {
            "symptom": "itchy, flaky and slightly scaly",
            "affected_area": ""
        }
    ],
    "current_meds": [
        {
            "medication": "OTC STEROID CREAM",
            "response": "moderate response"
        }
    ],
    "miscellaneous": "patient has been using cream for 2 weeks and also suffers from diabetes"
}

In [7]:
# Let's see the logs
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Given the following doctor's notes about a patient, please extract a dictionary that contains the       │ │
    │ │ patient's information.                                                                                  │ │
    │ │                                                                                                         │ │
    │ │ 152 y/o female with chronic macular rash to face and hair, worse in beard, eyebrows and nares.          │ │
    │ │ The rash is itchy, flaky and slightly scaly. Moderate response to OTC steroid cream. Patient has been   │ │
    │ │ using cream for 2 weeks and also suffers from diabetes.                                                 │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <string name="gender" description="Patient's gender"/>                                              │ │
    │ │     <integer name="age" description="Patient's age" format="valid-range: min=0 max=100"/>               │ │
    │ │     <list name="symptoms" description="Symptoms that the patient is currently experiencing. Each        │ │
    │ │ symptom should be classified into  separate item in the list.">                                         │ │
    │ │         <object>                                                                                        │ │
    │ │             <string name="symptom" description="Symptom that a patient is experiencing"/>               │ │
    │ │             <string name="affected_area" description="What part of the body the symptom is affecting"   │ │
    │ │ format="lower-case"/>                                                                                   │ │
    │ │         </object>                                                                                       │ │
    │ │     </list>                                                                                             │ │
    │ │     <list name="current_meds" description="Medications the patient is currently taking and their        │ │
    │ │ response">                                                                                              │ │
    │ │         <object>                                                                                        │ │
    │ │             <string name="medication" description="Name of the medication the patient is taking"        │ │
    │ │ format="upper-case"/>                                                                                   │ │
    │ │             <string name="response" description="How the patient is responding to the medication"/>     │ │
    │ │         </object>                                                                                       │ │
    │ │     </list>                                                                                             │ │
    │ │     <string name="miscellaneous" description="Any other information that is relevant to the patient's   │ │
    │ │ health; something that doesn't fit into the

As you can see here, the outputs in both examples match. The only difference is that, in the streaming example, the outputs are returned as soon as they are received and validated by Guardrails. In the non-streaming example, the outputs are returned only after the entire request has been processed by the API. In other words, when streaming is enabled, the API returns the outputs as soon as they are ready, rather than waiting for the entire request to be processed.


### 2. For unstructured text output


#### Define the prompt and Guard object with validators


In [8]:
from guardrails.validators import UpperCase, OneLine

prompt = """
Generate a short description of large language models. Each new sentence should be on another line.
"""

guard = gd.Guard.from_string(
    validators=[
        UpperCase(on_fail="fix"),
        OneLine(on_fail="fix"),
    ],
    description="testmeout",
    prompt=prompt,
)

#### Example 1: No streaming

By default, the `stream` parameter is set to `False`


In [9]:
# Wrap the OpenAI API call with the `guard` object
raw, validated, *rest = guard(
    openai.completions.create,
    engine="text-davinci-003",
    max_tokens=50,
    temperature=0.1,
)

# Print the raw and validated outputs
print(f"Raw output:\n{raw}")
print(f"Validated output:\n{validated}")

Raw output:
Large language models are powerful tools for natural language processing.
They are used to generate text, identify sentiment, and classify documents.
They are trained on large datasets and use deep learning algorithms to learn the language.
They can be used to generate

Validated output:
LARGE LANGUAGE MODELS ARE POWERFUL TOOLS FOR NATURAL LANGUAGE PROCESSING.

#### Example 2: With streaming

Set the `stream` parameter to `True`


In [10]:
# Wrap the OpenAI API call with the `guard` object
fragment_generator = guard(
    openai.completions.create,
    engine="text-davinci-003",
    max_tokens=50,
    temperature=0.1,
    stream=True,
)


for op in fragment_generator:
    clear_output(wait=True)
    print(op)
    time.sleep(0.1)

Raw LLM response:
Large language models are powerful tools for natural language processing tasks such as text classification, machine
translation, and question answering.
They are trained on large datasets of text and use deep learning techniques to learn the relationships between 
words and phrases.
They

Validated response:
"LARGE LANGUAGE MODELS ARE POWERFUL TOOLS FOR NATURAL LANGUAGE PROCESSING TASKS SUCH AS TEXT CLASSIFICATION, 
MACHINE TRANSLATION, AND QUESTION ANSWERING."

In [11]:
# See guard history
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Generate a short description of large language models. Each new sentence should be on another line.     │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ String Output:                                                                                          │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Message History ────────────────────────────────────────────╮ │
    │ │ No message history.                                                                                     │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ Large language models are powerful tools for natural language processing tasks such as text             │ │
    │ │ classification, machine translation, and question answering.                                            │ │
    │ │ They are trained on large datasets of text and use deep learning techniques to learn the relationships  │ │
    │ │ between words and phrases.                                                                              │ │
    │ │ They                                                                                                    │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ 'LARGE LANGUAGE MODELS ARE POWERFUL TOOLS FOR NATURAL LANGUAGE PROCESSING TASKS SUCH AS TEXT            │ │
    │ │ CLASSIFICATION, MACHINE TRANSLATION, AND QUESTION ANSWERING.'                                           │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

As you can see, the outputs in both examples match. The only difference is that, in the streaming example, the outputs are returned as soon as they are received and validated by Guardrails. In the non-streaming example, the outputs are returned only after the entire request has been processed by the API. In other words, when streaming is enabled, the API returns the outputs as soon as they are ready, rather than waiting for the entire request to be processed.

This also works with openai's chat completion API.
